In [2]:
# import Librarairs
import os,json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

CWD = os.getcwd()  # get current directory path
BUG_REPORTS_DIR = os.path.join(CWD,"bugs")   # current directory+bugs
BUG_REPORTS_LIST = os.listdir(BUG_REPORTS_DIR)  # all the items in 'current_directory+bugs'

SENTI_REPORTS_DIR = os.path.join(CWD,"Sentiments_output")
SENTI_REPORTS_LIST = os.listdir(SENTI_REPORTS_DIR)


In [3]:
# Load data set for bugs and sentiments

print(BUG_REPORTS_DIR)
print('-----------------')
print(BUG_REPORTS_LIST)
print('-----------------')
print(SENTI_REPORTS_DIR)
print('-----------------')
print(SENTI_REPORTS_LIST)   

/home/cr/Desktop/bugs
-----------------
['Core.json', 'Toolkit.json', 'bugzilla.json', 'Calendar.json', 'SeaMonkey.json', 'Camino Graveyard.json', 'MailNews Core.json', 'Firefox.json', 'Thunderbird.json', 'Core Graveyard.json']
-----------------
/home/cr/Desktop/Sentiments_output
-----------------
['Camino_Graveyard_sentiments.csv', 'bugzilla_sentiments.csv', 'Firefox_sentiments.csv', 'Core_sentiments.csv', 'MailNews_Core_sentiments.csv', 'SeaMonkey_sentiments.csv', 'Thunderbird_sentiments.csv', 'Toolkit_sentiments.csv', 'Core_Graveyard_sentiments.csv', 'Calendar_sentiments.csv']


In [4]:
# Extract data from files for sentiments
import csv
test_senti = []
train_senti=[]
import pandas as pd
for count, report in enumerate(SENTI_REPORTS_LIST):
    
    if count == 1:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data['neutral'])
            test_senti.extend(temp_data.iloc[0:, 1])
            
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data.iloc[0:, 1])
            train_senti.extend(temp_data.iloc[0:, 1])
            
            #print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")

Train Set: Camino_Graveyard_sentiments.csv
1291
Test Set: bugzilla_sentiments.csv
5325
Train Set: Firefox_sentiments.csv
7369
Train Set: Core_sentiments.csv
7229
Train Set: MailNews_Core_sentiments.csv
2797
Train Set: SeaMonkey_sentiments.csv
8717
Train Set: Thunderbird_sentiments.csv
8717
Train Set: Toolkit_sentiments.csv
1900
Train Set: Core_Graveyard_sentiments.csv
889
Train Set: Calendar_sentiments.csv
1786


In [5]:
# Load data for bugs 
train_set = []
test_set = []

for count, report in enumerate(BUG_REPORTS_LIST):
    if count == 0:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            test_set.extend(temp_data["bugs"])
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            train_set.extend(temp_data["bugs"])   

Test Set: Core.json
File:Core.json <> Reports: 7228
Train Set: Toolkit.json
File:Toolkit.json <> Reports: 1899
Train Set: bugzilla.json
File:bugzilla.json <> Reports: 5324
Train Set: Calendar.json
File:Calendar.json <> Reports: 1785
Train Set: SeaMonkey.json
File:SeaMonkey.json <> Reports: 8716
Train Set: Camino Graveyard.json
File:Camino Graveyard.json <> Reports: 1290
Train Set: MailNews Core.json
File:MailNews Core.json <> Reports: 2796
Train Set: Firefox.json
File:Firefox.json <> Reports: 7368
Train Set: Thunderbird.json
File:Thunderbird.json <> Reports: 5129
Train Set: Core Graveyard.json
File:Core Graveyard.json <> Reports: 888


In [6]:
summary_train=[]
label_train=[]
summary_test=[]
label_test=[]
for i in train_set:
    summary_train.append(i['summary'])
    #summary_train=summary_train[0:10]
    label_train.append(i['resolution'])
print('train_summary',len(summary_train))
print('train_label',len(label_train))

    #label_train=label_train[0:10]
for j in test_set:
    summary_test.append(j['summary'])
    #summary_test=summary_test[0:10]
    label_test.append(j['resolution'])
   # label_test=label_test[0:10]
print('test_summary',len(summary_test))
print('test_label',len(label_test))

train_summary 35195
train_label 35195
test_summary 7228
test_label 7228


In [7]:
def senti_to_int(train_test_senti):
    
    train_test_sentiments=[]
    for x in range(len(train_test_senti)):
        if train_test_senti[x]=="positive":
            train_test_senti[x]=1
        elif train_test_senti[x]=="neutral":
            train_test_senti[x]=1
        elif train_senti[x]=="negative":
            train_test_senti[x]=0
        train_test_sentiments.append(train_test_senti[x])
    return train_test_sentiments

In [8]:
#Preprocess sentences
import re
def train_test_summary(summaries_text):
    
    lemmas=[]
    lemmatized_word=[]
    train_test_summaries=[]
    for count, summaries in enumerate (summaries_text):
        text = re.sub(r'[^a-zA-Z\']', ' ', summaries)
        #removes all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        #substitute multiple spcaces with single space
        s_m_s = re.sub(r'\s+', ' ', document, flags=re.I)
        #removes prefix
        r_prefix = re.sub(r'^b\s+', '', s_m_s)
        #lemmas.append(r_prefix)
        r_pm = re.sub(r'[^\w\s]', '', r_prefix)
        step1 = r_pm.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
        step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
        step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
        step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
        step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
        step6 = step5.replace(" ` ", " '")
        step7=step6.strip()
        step8=step7.lower()    
        lemmas=[]
        sentences = nltk.word_tokenize(step7)
        lemmatizer = WordNetLemmatizer()
        stem_words=[]
        for sentence_words in sentences:
            lower=sentence_words.lower()
            if lower not in stopwords.words('english'):
                lemma = lemmatizer.lemmatize(lower)
                stem_words.append(lemma)
            untokenized_sentence= ' '.join(stem_words)
        train_test_summaries.append(untokenized_sentence)
    return train_test_summaries

In [9]:
# Encocede Labels 
def train_test_labels(labels):
    train_test_label=[]
    for x in range(len(labels)):
        
        
        if label_train[x]=="FIXED":
            label_train[x]=1
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="INVALID":
            label_train[x]=0
        elif label_train[x]=="INCOMPLETE":
            label_train[x]=0
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="WORKSFORME":
            label_train[x]=0
        elif label_train[x]=="EXPIRED":
            label_train[x]=0
        elif label_train[x]=="DUPLICATE":
            label_train[x]=0
        elif label_train[x]=="":
            label_train[x]=0
        train_test_label.append(label_train[x])
        
    
    
    #print('train_lbel', len(train_label))
    return train_test_label
    


In [10]:
train_labels= train_test_labels(label_train)
test_labels=train_test_labels(label_test)
train_summary=train_test_summary(summary_train)
test_summary=train_test_summary(summary_test)

train_sentiment=senti_to_int(train_senti)
test_sentiment=senti_to_int(test_senti)

# Sentiments 

train_senti=np.array(train_sentiment)
test_senti=np.array(test_sentiment)
#train_sentiments.shape
train_sentiments=train_senti.reshape(len(train_senti),1)
test_sentiments=test_senti.reshape(len(test_senti),1)


In [11]:
print(f"Training summary Length:{len(train_summary)}")
print(f"Testing summary Length:{len(test_summary)}")
print(f"Training Labels Length:{len(train_labels)}")
print(f"Testing Labels Length:{len(test_labels)}")
print(f"Training sentimetns Length:{len(train_sentiment)}")
print(f"Testing sentiments Length:{len(test_sentiment)}")


# Use 
#     train_labels
#     test_labels

# Use 
#     train_summary
#     test_summary

# Use 
#     train_sentiments
#     test_sentiments


Training summary Length:35195
Testing summary Length:7228
Training Labels Length:35195
Testing Labels Length:7228
Training sentimetns Length:40695
Testing sentiments Length:5325


In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [13]:
NUM_WORDS=1000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(train_summary)
sequences_train = tokenizer.texts_to_sequences(train_summary)
sequences_valid=tokenizer.texts_to_sequences(test_summary)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 10612 unique tokens.


In [17]:
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])
y_train = to_categorical(np.asarray(train_labels))
y_val = to_categorical(np.asarray(test_labels))
print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (35195, 28) (7228, 28)
Shape of label train and validation tensor: (35195, 2) (7228, 2)


In [19]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('/home/cr/Desktop/GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

W0902 00:01:05.127656 140186585446208 smart_open_lib.py:378] this function is deprecated, use smart_open.open instead


In [ ]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5



inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)


W0902 00:01:45.936730 140186585446208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 00:01:45.940925 140186585446208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0902 00:01:45.960899 140186585446208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0902 00:01:45.962442 140186585446208 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [1]:
adam = Adam(lr=1e-3)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='val_loss')]
model.fit(X_train, y_train, batch_size=1000, epochs=10, verbose=1, validation_data=(X_val, y_val),
         callbacks=callbacks)  # starts training

NameError: name 'Adam' is not defined

In [ ]:
sequences_test=tokenizer.texts_to_sequences(test_data.text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
y_pred=model.predict(X_test)

In [ ]:
yhat_class = (y_pred > 0.5).astype("int32")

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
print(classification_report(y_true=test_labels, y_pred=yhat_class, digits=5))